In [9]:
import cv2
import imutils as im
import numpy as np
import matplotlib.pyplot as plt
import keras_ocr
import os
import math


In [18]:
# Folder gambar untuk input dan folder untuk menyimpan hasil
uploads_dir = "D:/UTY/Semester 7/Pengembangan Aplikasi AI/Dataset/K1/JCM/"
uploads_dir_2 = "D:/UTY/Semester 7/Pengembangan Aplikasi AI/Dataset/K1/JIH/"
uploads_dir_3 = "D:/UTY/Semester 7/Pengembangan Aplikasi AI/Dataset/K1/Kost/"
save_dir = "D:/UTY/Semester 7/Pengembangan Aplikasi AI/Dataset/K1/Save/"

# Untuk membaca file satu per satu
# input = ""
# image = cv2.imread(input)

# Input weight Keras-OCR hasil training
recognizer = keras_ocr.recognition.Recognizer()
recognizer.model.load_weights("D:/UTY/Semester 7/Pengembangan Aplikasi AI/Dataset/K1/RSA/weight_file.h5")
pipeline = keras_ocr.Pipeline(recognizer=recognizer)


Looking for C:\Users\Aqil Wahid\.keras-ocr\crnn_kurapan.h5


FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = 'D:/UTY/Semester 7/Pengembangan Aplikasi AI/Dataset/K1/RSA/weight_file.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
#Fungsi untuk mengurutkan 4 edges pada plat agar konsisten
def order_points(pts):
    # Koordinat akan di urutkan dengan urutan kiri-atas 
    # #kanan-atas, kanan-bawah dan kiri-bawah
    rect = np.zeros((4, 2), dtype="float32")
    # Titik kiri-atas akah memiliki jumlah terkecil 
    # dan titik kanan-bawah memiliki jumlah terbesar
    s = pts.sum(axis = 1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]
    
    # Titik kanan-atas akan memiliki hasil pengurangan terkecil 
    # dan titik kiri-bawah memiliki hasil pengurangan terbesar
    diff = np.diff(pts, axis= 1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]
    # return koordinat yang sudah diurutkan
    return rect

#Fungsi untuk trasformasi gambar dengan mengambil gambar didalam area 4 titik
def four_point_transform(image, pts):
    (t1, tr, br, bl) = pts
    # Menghitung lebar gambar maksimum
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1])) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0])** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int (widthA), int (widthB))
    
    # Menghitung tinggi gambar maksimum
    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((t1[1] - bl[1]) ** 2))
    maxHeight = max(int (heightA), int(heightB))
    
    # Transformasi gambar dengan mengambil 4 titik untuk medapatkan 
    # tampak atas dari area gamber yang diingin
    dst = np.array([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
        [0, maxHeight - 1]], dtype="float32")
    
    #Menghitung Matriks Transformasi
    M = cv2.getPerspectiveTransform(pts, dst)
    warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))

    # Return gambar yang sudah di transformasi
    return warped

# Fungsi untuk menghitung sudut maksimal garis horizontal dari \
#Edges plat yang terdeteksi
def count_angle(points):
    atas_kiri, atas_kanan, bawah_kanan, bawah_kiri = points
    #Menghitung lebar dan tinggi di sepasang sisi
    height_1 = (atas_kanan[1]-atas_kiri[1])
    width_1 = (atas_kanan[0]-atas_kiri[0])
    height_2 = (bawah_kanan [1]-bawah_kiri[1])
    width_2 = (bawah_kanan[0]-bawah_kiri[0])
    
    # Mencari sudut maksimum relatif terhadap garis horizontal 
    # Sudut positif arah jarum jam dari sumbu x positif
    degree_1 = math.atan2 (height_1, width_1)
    degree_2 = math.atan2(height_2,width_2)
    if degree_1>=0 and degree_2>=0 :
        degree = max(degree_1, degree_2)
    elif degree_1<0 and degree_2>=0 :
        degree = degree_1
    elif degree_1>=0 and degree_2<0 :
        degree = degree_2
    elif degree_1<0 and degree_2<0 :
        degree = -1*(max (abs (degree_1), abs (degree_2)))
    else:
        degree = 0
    
    # mengubah dari unit radian ke derajat
    pi = math.pi
    degree_in_degree = (degree/(2*pi)*360)
    #return sudut dalam satuan derajat 
    return degree_in_degree

# Fungsi untuk lokalisasi kandidat plat kendaraan
# input berupa gambar grayscale yang sudah di inverse (karena plat indonesia 
# memiliki karakter cerah dengan latar gelap, algoritma ini awalnya dibuat untuk
# karakter gelap pada latar cerah
def locate_license_plate_candidates (gray, keep=3):
    # Melakukan blackhat morphological operation untuk menonjolkan area gelap
    # pada latar cerah
    rectKern = cv2.getStructuringElement(cv2.MORPH_RECT, (13, 5))
    blackhat = cv2.morphologyEx(gray, cv2.MORPH BLACKHAT, rectKern)
    
    # Mencari area terang pada gambar dengan close morphological operation 
    squareKern = cv2.getStructuringElement (cv2.MORPH_RECT, (3, 3))
    light = cv2.morphologyEx(gray, cv2.MORPH CLOSE, squareKern)
    # light cv2.cvtColor(light, cv2.COLOR_BGR2GRAY)
    light = cv2.threshold (light, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
    # Mempertajam fitur gradient edges dengan metode scharr pada sumbu x 
    # Kemudian mengembalikan range nilai piksel ke [8,255]
    gradX = cv2.Sobel (blackhat, ddepth = cv2.CV_32F,
                       dx=1, dy=0, ksize=-1)
    gradX = np.absolute(gradX)
    (minVal, maxVal) = (np.min(gradX), np.max(gradX))
    gradX = 255 * ((gradX - minVal) / (maxVal - minVal))
    gradX = gradX.astype("uint8")

    # Melakukan blur gauss pada gambar kemudian melakukan closing morphological
    # operation dan kemudian melakukan tresholding dengan metode otsu
    gradX = cv2.GaussianBlur (gradX, (5, 5), 0)
    gradX = cv2.morphologyEx (gradX, cv2.MORPH_CLOSE, rectKern)
    # gradX = cv2.cvtColor(gradX, cv2.COLOR_BGR2GRAY)
    thresh = cv2.threshold (gradX, 8, 255,
                            cv2.THRESH_BINARY | cv2.THRESH_OTSU) [1]

    # Melakukan operasi erode dan dilate untuk membuang noise diluar objek signifikan 
    thresh = cv2.erode (thresh, None, iterations=3)
    thresh = cv2.dilate (thresh, None, iterations=3)
    
    # Melakukan masking dengan gambar light hasil fungsi sebelumnya untuk
    # Memfokuskan gambar pada bagian cerah (tempat plat berada)
    # Kemudian melakukan operasi dilate dan erode untuk membuang noise dalam 
    # objek signifikan
    thresh = cv2.bitwise_and (thresh, thresh, mask=light)
    thresh = cv2.dilate (thresh, None, iterations=5)
    thresh = cv2.erode (thresh, None, iterations=5)
    # Mencari kontur tertutup dari gambar
    cnts,hir = cv2.findContours(thresh.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    cnts = sorted(cnts, key = cv2.contourArea, reverse = True)
    
    #Template untuk masking
    mask = np.ones(image.shape[:2], dtype="uint8") * 255

    # print(len(cnts))
    if len(cnts) >= 4:
        cv2.drawContours (mask, cnts[4:], -1, 0, -1)
    thresh = cv2.bitwise and (thresh, thresh, mask-mask)
    # else :
    #   cv2.drawContours (mask, cnts[len(cnts):], -1, 0, -1)

    # Masking untuk kontur yang memiliki ukuran kecil dibanding kontur
    # objek utama
    for c in cnts :
        if cv2.arcLength(c, True) < 0.25*cv2.arcLength (cnts[0], True) :
            cv2.drawContours (mask, [c], -1, 0, -1)
    thresh = cv2.bitwise_and (thresh, thresh, mask=mask)
    cnts,hir = cv2.findContours (thresh.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    cnts = sorted(cnts, key = cv2.contourArea, reverse = True)
    
    # Membuat boundary box dari semua objek yang diperkirakan sebagai plat 
    # kendaraan pada gambar binary
    points = cv2. findNonZero(thresh)
    xandy, dimension, degree = cv2.minAreaRect(points)
    dimension = list(dimension)
    dimension[0] = 1*dimension[0]+10
    dimension[1] = 1*dimension[1]+10
    rect = []
    rect.append(xandy)
    rect.append(dimension)
    rect.append(degree)
    rect = tuple(rect)
    
    # Variable untuk menyimpan koordinat 4 titik sudut dari prediksi bounding box
    # untuk plat kendaraan
    box = cv2.boxPoints (rect)
    box = np.int0 (box)
    
    # return kooerdinat dan gambar hasil lokalisasi
    return box, thresh

# Algoritma untuk mencari 4 titik sudut dari lokalisasi plat nomor
def closed_contour_method(image) :
    # Konversi gambar menjadi abu-abu
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # # Menghilangkan noise dengan iterative bilateral filter
    #d, sigmaColor, sigmaSpace 5,15,15
    # filtered_img = cv2.bilateralFilter (gray, d, sigmaColor, sigmaSpace)
    
    # Menghilangkan noise dengan GaussianBlur
    filtered_img = cv2.GaussianBlur (gray, (5,5),0)
    
    # Mencari edges dari dambar dengan algoritma canny
    lower, upper = 50, 150
    edged = cv2.Canny(filtered_img, lower, upper)
    
    # Mencari kontur tertutup dari gambar kemudian diurutkan dari yang terbesar
    cnts, hir = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE) 
    cnts = sorted(cnts, key = cv2.contourArea, reverse= True)[:10]
    NumberPlateCnt = np.zeros((4,2))
    
    # Mencari kontur yang memiliki potensi sebagai objek plat
    count = 8
    for c in cnts:
        peri = cv2.arcLength(c, True)
        # Aproksimasi panjang kontur yang berpotensi
        epsilon = 0.01 * peri
        approx = cv2.approxPolyDP (c, epsilon, True)
        
        # Memilih kontur dengan 4 titik (quadrilateral)
        if len(approx) == 4:
            NumberPlateCnt = approx
            NumberPlateCnt = NumberPlateCnt.reshape(4,2)
            count += 1
            break
    
    return NumberPlateCnt